#  Homework 2 Analysis

## Summary 
Forecast Colorado state marijuana tax revenue for calendar years 2024 and 2025. 
While background information is provided here, please also rely on news articles and other sources of information when preparing your forecast. 

Tax revenue data are provided along with this assignment and are summarized on a monthly and annual basis. In three pages or less, provide your forecast estimates (e.g., $\$XXX.X$ million and $\$XXX.X$ million) for calendar years 2024 and 2025 and summarize your forecast by providing an economic narrative substantiating these expectations. 

This narrative should identify key drivers of tax revenue (e.g., determinants of demand, supply, prices, and/or other factors) and your expectations for changes in these drivers.

## Background and other considerations

### 1. Marijuana taxes and tax rates. 
Three unique taxes apply differently to medical and retail marijuana. 

|              | Regular State Sales Tax |  Marijuana Retail Excise Tax | Marijuana Retail Sales Tax | 
|--------------|-------------------------|------|--------|
| Description  | Tax on most goods sold in Colorado. While medical marijuana sales are subject to the tax, retail marijuana is exempt. | Tax on the first sale or transfer of unprocessed marijuana by a retail cultivation facility. | Tax on the sale of retail marijuana or retail marijuana products to a consumer by a retail marijuana store. |
| Tax Rate | 2.9% Beginning July 1, 2017, only marijuana accessories subject to the tax, while marijuana and marijuana products are exempt. | 15% | Increased from 10% to 15% beginning July 1, 2017. |

2. Median market price data are provided with tax revenue data.

4. Legalization and taxation of marijuana in other states.
    - For a summary of states that have legalized marijuana, see: [NCSL Org](https://www.ncsl.org/research/health/state-medical-marijuana-laws.aspx).
    - For a summary of marijuana taxation across states, see: [Tax Foundation](https://taxfoundation.org/research/all/state/cannabis-tax-revenue-reform/).


## Literature

* "Marijuana Policy Overview, FY 2024-2025", Joint Budget Committee, Colorado LegislatureE  * [Link](W
https://leg.colorado.gov/sites/default/files/fy2024_-25marbrf.p)df

## Approach and Methodology

Marijuana is an interesting product which I am note sure how to categorize.  To start with, I will make the assumption that Marijuana products will likely follow the market as a whole.  

# Preparation Work

First, we load the require Python libraries and setup our logger.

In [1]:
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px

import matplotlib.pyplot as plt

import numpy as np

from ipywidgets import interact

from sklearn.linear_model import LinearRegression

import datetime, dateutil, logging, os, sys

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

ModuleNotFoundError: No module named 'ipywidgets'

Logger setup

In [ ]:
logging.basicConfig( level = logging.INFO )

In [ ]:
num_dims = 2

In [ ]:
tax_rev_monthly_df = pd.read_excel( 'Assignment 2 - Data.xlsx',
                                    sheet_name = 'tax_revenue_monthly' )

tax_rev_monthly_df.head(3)

In [ ]:
#dmonth.dt.components.days.values.shape
tax_rev_monthly_df['reg_state_sales_tax'].values.shape

In [ ]:
tax_rev_yearly_df = pd.read_excel( 'Assignment 2 - Data.xlsx',
                                   sheet_name = 'tax_revenue_yearly' )

tax_rev_yearly_df.head(3)

In [ ]:
median_price = pd.read_excel( 'Assignment 2 - Data.xlsx',
                              sheet_name = 'median_market_price' )
median_price.head(3)

In [ ]:
fig = go.Figure()

fig.add_trace( go.Scatter( x = tax_rev_monthly_df['date'],
                           y = tax_rev_monthly_df['reg_state_sales_tax'],
                           name = 'Regular State Sales Taxes' ) )

fig.add_trace( go.Scatter( x = tax_rev_monthly_df['date'],
                           y = tax_rev_monthly_df['ret_mar_sales_tax'],
                           name = 'Retail Marijuana Sales Taxes' ) )

fig.add_trace( go.Scatter( x = tax_rev_monthly_df['date'],
                           y = tax_rev_monthly_df['ret_mar_exc_tax'],
                           name = 'Retail Marijuana Excise Taxes' ) )
fig.update_layout( height = 500 )

fig.show()

In [ ]:
start_date = tax_rev_monthly_df['date'].values[0]

In [ ]:
date_range = []
date_range_td = []
for year in range(2014,2027):
    for month in range( 1,13 ):
        date_range.append( np.datetime64( f'{year}-{month:02}-01' ) )
        date_range_td.append( np.datetime64( f'{year}-{month:02}-01' ) - tax_rev_monthly_df['date'].values[0])

date_range_dt = pd.Series( date_range )
date_range_td_dt = pd.Series( date_range_td ).dt.days

In [ ]:
dmonth = tax_rev_monthly_df['date'] - tax_rev_monthly_df['date'].values[0]

In [ ]:
res = np.polyfit( dmonth.dt.days,
                  tax_rev_monthly_df['reg_state_sales_tax'].values,
                  num_dims )
print(res)

In [ ]:
print(dmonth.dtype,date_range_dt.dtype, date_range_td_dt.dtype )

In [ ]:
y1 = np.polyval( res, date_range_td_dt) #month.values.astype('float64') )

In [ ]:
poly = PolynomialFeatures(degree=num_dims, include_bias=False)
poly_features = poly.fit_transform( dmonth.dt.days.values.reshape(-1, 1) )

In [ ]:
#fit polynomial regression model
poly_reg_model = LinearRegression()

#print( poly_features.shape, len(date_range_val3 ), len(date_range_dt))
poly_reg_model.fit(poly_features, tax_rev_monthly_df['reg_state_sales_tax'] ) #dmonth.dt.days.values.astype('float64').reshape(-1, 1))
print(poly_reg_model)

#use model to make predictions on response variable
y2 = poly_reg_model.predict( poly.fit_transform( date_range_td_dt.values.astype('float64').reshape(-1,1))).reshape(date_range_dt.shape[0])

In [ ]:
print(len(y1),len(y2),type(y1),type(y2), y1.dtype, y2.dtype,y1.shape,y2.shape)

In [ ]:
fig2 = go.Figure()

fig2.add_trace( go.Scatter( x = date_range,
                            y = tax_rev_monthly_df['reg_state_sales_tax'],
                            name = 'State Sales Tax' ) )

fig2.add_trace( go.Scatter( x = date_range_dt,
                            y = y1,
                            name = 'Polyfit Line' ) )


fig2.add_trace( go.Scatter( x = date_range_dt,
                            y = y2,
                            name = 'Regression Line' ) )

fig2.update_xaxes( range = [np.datetime64('2014-01-01'),np.datetime64('2026-01-01')] )
#fig2.update_yaxes( range = [0,tax_rev_monthly_df['reg_state_sales_tax'].max()] )
fig2.update_layout( height = 500 )
fig2.show()